In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory,BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.chat_message_histories import ChatMessageHistory

In [16]:
llm = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=4096)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个智能聊天机器人，名字叫二狗，请用markdown格式回答任何问题",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chain = prompt | llm


In [17]:
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [18]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [19]:

res = with_message_history.invoke(
    {"input": "我刚说过什么？？"},
    config={"configurable": {"session_id": "111"}}
)
res

Parent run 211b4f22-547b-4b12-a1b6-703bfe4984e0 not found for run 8160f67d-1943-4e8b-9213-603894309650. Treating as a root run.


AIMessage(content='你刚说的是：“我刚说过什么？？”', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 52, 'total_tokens': 69}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-916f3ffe-4585-4d86-8466-b3960d13ef04-0')

In [2]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    return SQLChatMessageHistory(
        session_id=session_id, connection_string="sqlite:///langchain.db"
    )


In [5]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [6]:
res = with_message_history.invoke(
    {"input": "我刚说过什么？？"},
    config={"configurable": {"session_id": "111"}}
)
res

Parent run e1da6d41-e78f-4f4b-814a-8e001207ccec not found for run 30eb9a65-ecbd-4eb0-9ad6-a4eec83fa4be. Treating as a root run.


In [7]:
history = get_session_history("111")

In [8]:
history.messages

[HumanMessage(content='我刚说过什么？？'),
 AIMessage(content='你刚说：“我刚说过什么？？”', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 52, 'total_tokens': 67}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1260d9af-8731-486a-a29d-061e42e6e219-0')]